In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [41]:
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]
import pandas as pd
# Load the dataset
df = pd.read_csv('adult.data', names=column_names)
df = df[column_names]
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [42]:
df["sex"].value_counts()

,count
sex,
Male,21790
Female,10771


In [43]:
# Preprocessing: Convert categorical columns to numerical (using One-Hot Encoding)
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country']
df['sex'] = df['sex'].apply(lambda x: 1 if x.strip() == 'Male' else 0)
df['race'] = df['race'].apply(lambda x: 1 if x.strip() == 'White' else 0)
df = pd.get_dummies(df, columns=categorical_columns)  # One-hot encode categorical columns

In [44]:
print(df.columns)

Index(['age', 'fnlwgt', 'education-num', 'race', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'income', 'workclass_ ?',
       ...
       'native-country_ Portugal', 'native-country_ Puerto-Rico',
       'native-country_ Scotland', 'native-country_ South',
       'native-country_ Taiwan', 'native-country_ Thailand',
       'native-country_ Trinadad&Tobago', 'native-country_ United-States',
       'native-country_ Vietnam', 'native-country_ Yugoslavia'],
      dtype='object', length=104)


In [45]:
# Encode the target variable 'income' (<=50K -> 0, >50K -> 1)
le = LabelEncoder()
df['income'] = le.fit_transform(df['income'])

# Separate features and target variable
X = df.drop('income', axis=1)
y = df['income']

In [46]:
# Split the data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model on the training data
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [47]:
def evaluate_counterfactual_fairness_sex(model, X, target_column='income'):
    """
    Evaluates counterfactual fairness by flipping the 'sex' attribute.
    """
    # Store the original predictions on the test set
    original_predictions = model.predict(X)

    # Create a copy of the dataset
    X_counterfactual = X.copy()

    # Flip 'sex' (0 -> 1, 1 -> 0)
    if 'sex' in X.columns:
        X_counterfactual['sex'] = 1 - X_counterfactual['sex']
    else:
        raise KeyError(f"The sensitive attribute 'sex' is not present in the dataset.")

    # Get counterfactual predictions
    counterfactual_predictions = model.predict(X_counterfactual)

    # Compare the original and counterfactual predictions
    comparison = pd.DataFrame({
        'original': original_predictions,
        'counterfactual': counterfactual_predictions,
        'same_decision': original_predictions == counterfactual_predictions
    })

    # Return the comparison dataframe
    return comparison

# Run the counterfactual fairness evaluation on the 'sex' attribute using the test set
comparison_sex_df = evaluate_counterfactual_fairness_sex(model, X_test)

# Display the result
print("Counterfactual fairness evaluation for 'sex' (Male/Female):")
print(comparison_sex_df.head())

Counterfactual fairness evaluation for 'sex' (Male/Female):
   original  counterfactual  same_decision
0         0               0           True
1         0               0           True
2         1               1           True
3         0               0           True
4         0               0           True


In [49]:
# Calculate the fairness metric for sex (Male/Female)
print(comparison_sex_df["same_decision"].value_counts())
fairness_metric_sex = comparison_sex_df['same_decision'].mean()
print(f"Fairness metric for sex: {fairness_metric_sex:.4f}")

same_decision
True     6265
False     248
Name: count, dtype: int64
Fairness metric for sex: 0.9619


In [52]:
def evaluate_counterfactual_fairness_race(model, X, target_column='income'):
    """
    Evaluates counterfactual fairness by flipping the 'race' attribute.
    """
    # Store the original predictions on the test set
    original_predictions = model.predict(X)
    # Create a copy of the dataset
    X_counterfactual = X.copy()

    # Flip 'race' (0 -> 1, 1 -> 0)
    if 'race' in X.columns:
        X_counterfactual['race'] = 1 - X_counterfactual['race']
    else:
        raise KeyError(f"The sensitive attribute 'race' is not present in the dataset.")

    # Get counterfactual predictions
    counterfactual_predictions = model.predict(X_counterfactual)

    # Compare the original and counterfactual predictions
    comparison = pd.DataFrame({
        'original': original_predictions,
        'counterfactual': counterfactual_predictions,
        'same_decision': original_predictions == counterfactual_predictions
    })

    # Return the comparison dataframe
    return comparison

# Run the counterfactual fairness evaluation on the 'race' attribute using the test set
comparison_race_df = evaluate_counterfactual_fairness_race(model, X_test)

# Display the result
print("Counterfactual fairness evaluation for 'race' (White/Non-White):")
print(comparison_race_df)

Counterfactual fairness evaluation for 'race' (White/Non-White):
      original  counterfactual  same_decision
0            0               0           True
1            0               0           True
2            1               1           True
3            0               0           True
4            0               0           True
...        ...             ...            ...
6508         0               0           True
6509         0               0           True
6510         1               1           True
6511         0               0           True
6512         0               0           True

[6513 rows x 3 columns]


In [53]:
# Calculate the fairness metric for race (White vs Non-White)
print(comparison_race_df["same_decision"].value_counts())
fairness_metric_race = comparison_race_df['same_decision'].mean()
print(f"Fairness metric for race (White vs Non-White): {fairness_metric_race:.4f}")

same_decision
True     6183
False     330
Name: count, dtype: int64
Fairness metric for race (White vs Non-White): 0.9493
